In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact

In [2]:
alfa = 5

In [3]:
def test_ez (x: np.ndarray) -> np.ndarray:
    return np.sum((x-alfa)**2)

In [4]:
def test_ez_grad (x: np.ndarray) -> np.ndarray:
    return (x-alfa)*2

In [5]:
def test_rust (x: np.ndarray): # Функция Растригина
    A = 10
    return A * x.size + np.sum(x ** 2 - A * np.cos(2*np.pi*x))

In [6]:
def test_styb (x: np.ndarray): # Функция Стыбинского-Танга
    return np.sum(x ** 4 - 16 * x ** 2 + 5 * x)/2

# Визуализация

In [7]:
Acc = 500
x1 = np.linspace(-5, 5, Acc)
x2 = np.linspace(-5, 5, Acc)
Z = np.zeros((Acc, Acc))
Z2 = np.zeros((Acc, Acc))
for i in range (Acc):
    for j in range (Acc):
        Z[i, j] = test_rust(np.array([x1[i], x2[j]]))
        Z2[i, j] = test_styb(np.array([x1[i], x2[j]]))
        
def plot_3d_graph(angle_x, angle_y, alpha, x1, x2, Z, history1 = None, history2 = None):
    fig = plt.figure(figsize=(12, 6))  # Увеличьте ширину фигуры, чтобы разместить два графика рядом
    ax1 = fig.add_subplot(121, projection='3d')  # Создайте первую 3D-сетку
    ax2 = fig.add_subplot(122, projection='3d')  # Создайте вторую 3D-сетку

    

    X1, X2 = np.meshgrid(x1, x2)
    
    ax1.plot_surface(X1, X2, Z, alpha=alpha, cmap='viridis')
    if history1:
        history1 = np.array(history1)
        ax1.plot(history1[:, 0], history1[:, 1], history1[:, 2], '-', c='black', alpha = 1)
        ax1.scatter3D(history1[0, 0], history1[0, 1], history1[0, 2], s=250, c="white", lw=2, ec='black', marker = '.')
        ax1.scatter3D(history1[-1, 0], history1[-1, 1], history1[-1, 2], s=250, c="white", lw=2, ec='black', marker = '*')

    ax2.plot_surface(X1, X2, Z2, alpha=alpha, cmap='viridis')
    if history2:
        history2 = np.array(history2)
        ax2.plot(history2[:, 0], history2[:, 1], history2[:, 2], '-', c='black', alpha = 1)
        ax2.scatter3D(history2[0, 0], history2[0, 1], history2[0, 2], s=250, c="white", lw=2, ec='black', marker = '.')
        ax2.scatter3D(history2[-1, 0], history2[-1, 1], history2[-1, 2], s=250, c="white", lw=2, ec='black', marker = '*')
    
    ax1.view_init(elev=angle_x, azim=angle_y)
    ax2.view_init(elev=angle_x, azim=angle_y)

    plt.show()
graph = lambda angle_x, angle_y, alpha: plot_3d_graph(angle_x, angle_y, alpha, x1 = x1, x2 = x2, Z = Z)
# Создайте интерактивный виджет для прокрутки и вращения графика
interact(graph, angle_x=(-90, 90, 5), angle_y=(0, 360, 5), alpha=(0, 1, 0.1)) # Отрисовка Функции Растригина

interactive(children=(IntSlider(value=0, description='angle_x', max=90, min=-90, step=5), IntSlider(value=180,…

<function __main__.<lambda>(angle_x, angle_y, alpha)>

# Градиентные спуски

In [8]:
def num_diff(
    x: np.ndarray,
    use_function: bool = True, 
    h: float = 1e-7,
    **kwargs) -> np.ndarray:
    if use_function:
        f = kwargs['f']
    else:
        NotImplementedError()
    grad = np.zeros_like(x)
    
    for i in range(len(x)):
        original_value = x[i]
        
        # Вычисляем градиент по i-ой переменной с шагом h
        x[i] = original_value + h
        fx_plus_h = f(x)
        
        x[i] = original_value - h
        fx_minus_h = f(x)
        
        grad[i] = (fx_plus_h - fx_minus_h) / (2 * h)
        
        # Восстанавливаем исходное значение переменной x[i]
        x[i] = original_value
    
    return grad
    

In [9]:
def my_GD(
         x0: np.ndarray,
         T: int = 100,
         lr: float = 1e-3,
         grad_type: str = 'num_diff',
         use_crit: bool = False,
         crit: float = 1e-6,
         **kwargs) -> np.ndarray:    
    if grad_type != 'input' or grad_type != 'num_diff' or grad_type != 'sym_diff':
        grad_type = 'num_diff'
    if grad_type == 'input':
        df = kwargs['df']
    elif grad_type == 'num_diff':
        f = kwargs['f']
        df = lambda x: num_diff(x, f = f)
        
    x = x0
    lx = []
    lx.append(np.concatenate([x, [f(x)]]))
    if use_crit:
        while (np.sum(np.absolute(df(x))) > crit):
            x -= lr*df(x)
            lx.append(np.concatenate([x, [f(x)]]))
    else:
        for _ in range(T):
            x -= lr*df(x)
            lx.append(np.concatenate([x, [f(x)]]))
    return lx

In [10]:
def my_MGD(
         x0: np.ndarray,
         T: int = 100,
         lr: float = 1e-3,
         grad_type: str = 'num_diff',
         use_crit: bool = False,
         crit: float = 1e-6,
         beta: float = 0.5,
         **kwargs) -> np.ndarray:    
    if grad_type != 'input' or grad_type != 'num_diff' or grad_type != 'sym_diff':
        grad_type = 'num_diff'
    if grad_type == 'input':
        df = kwargs['df']
    elif grad_type == 'num_diff':
        f = kwargs['f']
        df = lambda x: num_diff(x, f = f)
    x_prev = x0.copy()
    x = x0.copy()    
    lx = []
    lx.append(np.concatenate([x, [f(x)]]))
    if use_crit:
        while (np.sum(np.absolute(df(x))) > crit):
            x_new = x - lr * df(x) + beta * (x - x_prev)
            x_prev = x
            x = x_new
            lx.append(np.concatenate([x, [f(x)]]))
    else:
        for _ in range(T):
            x_new = x - lr * df(x) + beta * (x - x_prev)
            x_prev = x
            x = x_new
            lx.append(np.concatenate([x, [f(x)]]))
    return lx

In [11]:
def my_ADAM(
         x0: np.ndarray,
         T: int = 100,
         lr: float = 1e-3,
         grad_type: str = 'num_diff',
         use_crit: bool = False,
         crit: float = 1e-6,
         b1: float = 0.9,
         b2: float = 0.999,
         e: float = 1e-3,
         **kwargs) -> np.ndarray:    
    if grad_type != 'input' or grad_type != 'num_diff' or grad_type != 'sym_diff':
        grad_type = 'num_diff'
    if grad_type == 'input':
        df = kwargs['df']
    elif grad_type == 'num_diff':
        f = kwargs['f']
        df = lambda x: num_diff(x, f = f)

    x = x0.copy()

    # инициализация моментов
    m = np.zeros_like(x)
    v = np.zeros_like(x)  
    lx = []
    lx.append(np.concatenate([x, [f(x)]]))
    if use_crit:
        while (np.sum(np.absolute(df(x))) > crit):
             # считаем моменты
            m = b1 * m + (1 - b1) * df(x)
            v = b2 * v + (1 - b2) * df(x) ** 2

        # новое значение параметров
            x = x - lr * m / (np.sqrt(v) + e)
            lx.append(np.concatenate([x, [f(x)]]))
    else:
        for _ in range(T):
             # считаем моменты
            m = b1 * m + (1 - b1) * df(x)
            v = b2 * v + (1 - b2) * df(x) ** 2

            # новое значение параметров
            x = x - lr * m / (np.sqrt(v) + e)
            lx.append(np.concatenate([x, [f(x)]]))
    return lx

# Обычный градиентный спуск

In [12]:
xt = np.random.normal(0, 1, 2)
h1 = my_GD(x0 = xt, use_crit = True, f = test_rust)
print(h1[-1])

[ 2.10159495e-09 -9.94958638e-01  9.94959057e-01]


In [13]:
xt = np.random.normal(0, 1, 2)
h2 = my_GD(x0 = xt, use_crit = True, f = test_styb)
print(h2[-1])



[  2.74680274   2.74680277 -50.05889331]


In [14]:
graph2 = lambda angle_x, angle_y, alpha: plot_3d_graph(angle_x, angle_y, alpha, x1 = x1, x2 = x2, Z = Z, history1=h1, history2=h2)
# Создайте интерактивный виджет для прокрутки и вращения графика
interact(graph2, angle_x=(-90, 90, 5), angle_y=(0, 360, 5), alpha=(0, 1, 0.1)) # Отрисовка Функции Растригина

interactive(children=(IntSlider(value=0, description='angle_x', max=90, min=-90, step=5), IntSlider(value=180,…

<function __main__.<lambda>(angle_x, angle_y, alpha)>

# Моментная модификация

In [15]:
xt = np.random.normal(0, 1, 2)
hm1 = my_MGD(x0 = xt, use_crit = True, f = test_rust)
print(h1[-1])

[ 2.10159495e-09 -9.94958638e-01  9.94959057e-01]


In [16]:
xt = np.random.normal(0, 1, 2)
hm2 = my_MGD(x0 = xt, use_crit = True, f = test_styb)
print(hm2[-1])

[ -2.90353403   2.74680274 -64.19561236]


In [17]:
graph3 = lambda angle_x, angle_y, alpha: plot_3d_graph(angle_x, angle_y, alpha, x1 = x1, x2 = x2, Z = Z, history1=hm1, history2=hm2)
# Создайте интерактивный виджет для прокрутки и вращения графика
interact(graph3, angle_x=(-90, 90, 5), angle_y=(0, 360, 5), alpha=(0, 1, 0.1)) # Отрисовка Функции Растригина

interactive(children=(IntSlider(value=0, description='angle_x', max=90, min=-90, step=5), IntSlider(value=180,…

<function __main__.<lambda>(angle_x, angle_y, alpha)>

# Адаптивная модификация

In [18]:
xt = np.random.normal(0, 1, 2)
ha1 = my_ADAM(x0 = xt, use_crit = True, f = test_rust)
print(ha1[-1])

[5.75425118e-12 9.94958636e-01 9.94959057e-01]


In [19]:
xt = np.random.normal(0, 1, 2)
ha2 = my_ADAM(x0 = xt, use_crit = True, f = test_styb)
print(ha2[-1])

[ -2.90353403  -2.903534   -78.33233141]


In [20]:
graph4 = lambda angle_x, angle_y, alpha: plot_3d_graph(angle_x, angle_y, alpha, x1 = x1, x2 = x2, Z = Z, history1=ha1, history2=ha2)
# Создайте интерактивный виджет для прокрутки и вращения графика
interact(graph3, angle_x=(-90, 90, 5), angle_y=(0, 360, 5), alpha=(0, 1, 0.1)) # Отрисовка Функции Растригина

interactive(children=(IntSlider(value=0, description='angle_x', max=90, min=-90, step=5), IntSlider(value=180,…

<function __main__.<lambda>(angle_x, angle_y, alpha)>